In [ ]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pickle

In [ ]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [ ]:
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [ ]:
class BinaryClassification(torch.nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        self.layer_1 = torch.nn.Linear(29, 64) 
        self.layer_2 = torch.nn.Linear(64, 64)
        self.layer_out = torch.nn.Linear(64, 1) 
        
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.1)
        self.batchnorm1 = torch.nn.BatchNorm1d(64)
        self.batchnorm2 = torch.nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
EPOCHS = 500
BATCH_SIZE = 10
LEARNING_RATE = 0.001
base_path = "/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/Bot-Iot"
X_train = np.genfromtxt(f"{base_path}/Sampled/30-sampled-x-train.csv", delimiter=',')
y_train = np.genfromtxt(f"{base_path}/Sampled/30-sampled-y-train.csv", delimiter=',')
X_test = np.genfromtxt(f"{base_path}/Sampled/30-sampled-x-test.csv", delimiter=',')
y_test = np.genfromtxt(f"{base_path}/Sampled/30-sampled-y-test.csv", delimiter=',')

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.astype("int64")
train_data = TrainData(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BinaryClassification()
model.to(device)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.08453 | Acc: 97.591
Epoch 002: | Loss: 0.02987 | Acc: 99.231
Epoch 003: | Loss: 0.02451 | Acc: 99.445
Epoch 004: | Loss: 0.02716 | Acc: 99.294
Epoch 005: | Loss: 0.01883 | Acc: 99.496
Epoch 006: | Loss: 0.01881 | Acc: 99.470
Epoch 007: | Loss: 0.01882 | Acc: 99.433
Epoch 008: | Loss: 0.01757 | Acc: 99.533
Epoch 009: | Loss: 0.01498 | Acc: 99.609
Epoch 010: | Loss: 0.01456 | Acc: 99.634
Epoch 011: | Loss: 0.01567 | Acc: 99.533
Epoch 012: | Loss: 0.01377 | Acc: 99.672
Epoch 013: | Loss: 0.01547 | Acc: 99.546
Epoch 014: | Loss: 0.01275 | Acc: 99.634
Epoch 015: | Loss: 0.01421 | Acc: 99.596
Epoch 016: | Loss: 0.01572 | Acc: 99.609
Epoch 017: | Loss: 0.01367 | Acc: 99.609
Epoch 018: | Loss: 0.01312 | Acc: 99.584
Epoch 019: | Loss: 0.01493 | Acc: 99.584
Epoch 020: | Loss: 0.01131 | Acc: 99.672
Epoch 021: | Loss: 0.01028 | Acc: 99.723
Epoch 022: | Loss: 0.01105 | Acc: 99.710
Epoch 023: | Loss: 0.01378 | Acc: 99.685
Epoch 024: | Loss: 0.01202 | Acc: 99.660
Epoch 025: | Los

In [ ]:
X_test_df = pd.DataFrame(X_test)
X_test_df.to_csv(f"{base_path}/Model/rnn-x_test.csv")
y_test_df = pd.DataFrame(y_test)
y_test_df.to_csv(f"{base_path}/Model/rnn-y_test.csv")
pickle.dump(model, open(f"{base_path}/Model/rnn-model.pkl", 'wb'))

In [ ]:
rate = 30
X_test = pd.read_csv(f"{base_path}/Model/rnn-x_test.csv", index_col=0)
y_test = pd.read_csv(f"{base_path}/Model/rnn-y_test.csv", index_col=0)
test_data = TestData(torch.FloatTensor(X_test.to_numpy()))
test_loader = DataLoader(dataset=test_data, batch_size=1)

y_pred_list = []
model = pickle.load(open(f"{base_path}/Model/rnn-model.pkl", 'rb'))
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
print(confusion_matrix(y_test, y_pred_list))
print(classification_report(y_test, y_pred_list))

[[2726    2]
 [   7 1172]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2728
         1.0       1.00      0.99      1.00      1179

    accuracy                           1.00      3907
   macro avg       1.00      1.00      1.00      3907
weighted avg       1.00      1.00      1.00      3907



In [ ]:
np.savetxt(f"{base_path}/Model/rnn-y-pred.csv", y_pred_list, delimiter=",")